<a href="https://colab.research.google.com/github/Herodegon/ECGR4105-Intro-to-Machine-Learning/blob/main/Assignment6_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [433]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [434]:
file_path = '/content/drive/MyDrive/ECGR4105/HW2_Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
M=len(dataset)
M

545

In [435]:
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


Problem 1 (40 pts):

1a. Develop a Fully Connected Neural Network with only one hidden layer (size of 32) to predict the housing value for the housing dataset. Make sure to include all input features. Compare your training loss value and validation results against the linear regression you implemented in Assignment 5. Can you compare your model complexity (number of trainable parameters) against linear regression?

  Note: Perform 20%, and 80% split for training and validation.

In [436]:
def FactorizeData(X):
  for i in range(0,X.shape[1]):
    for j in range(0,X.shape[0]):
      if X[j,i] == 'yes':
        X[j,i] = 2
      elif X[j,i] == 'no':
        X[j,i] = 1
      else:
        X[j,i] = X[j,i]
  return X

In [437]:
X1 = dataset.iloc[:,[1,2,3,4,5,6,7,8,9,10,11]].values                # Get input values from each explanatory variable
X1 = FactorizeData(X1).astype(float)
Y1 = dataset.iloc[:,0].values.astype(float)                          # Get output values (Price of Housing)

print("X_area = ", np.hstack(X1[:5,0]))
print("X_bedrooms = ", np.hstack(X1[:5,1]))
print("X_bathrooms = ", np.hstack(X1[:5,2]))
print("X_stories = ", np.hstack(X1[:5,3]))
print("X_mainroad = ", np.hstack(X1[:5,4]))
print("X_guestroom = ", np.hstack(X1[:5,5]))
print("X_basement = ", np.hstack(X1[:5,6]))
print("X_hotwaterheating = ", np.hstack(X1[:5,7]))
print("X_airconditioning = ", np.hstack(X1[:5,8]))
print("X_parking = ", np.hstack(X1[:5,9]))
print("X_prefarea = ", np.hstack(X1[:5,10]))
print("Y = ", np.hstack(Y1[:5]))

X_area =  [7420. 8960. 9960. 7500. 7420.]
X_bedrooms =  [4. 4. 3. 4. 4.]
X_bathrooms =  [2. 4. 2. 2. 1.]
X_stories =  [3. 4. 2. 2. 2.]
X_mainroad =  [2. 2. 2. 2. 2.]
X_guestroom =  [1. 1. 1. 1. 2.]
X_basement =  [1. 1. 2. 2. 2.]
X_hotwaterheating =  [1. 1. 1. 1. 1.]
X_airconditioning =  [2. 2. 1. 2. 2.]
X_parking =  [2. 3. 2. 3. 2.]
X_prefarea =  [2. 1. 2. 2. 1.]
Y =  [13300000. 12250000. 12250000. 12215000. 11410000.]


In [438]:
import torch as torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

t_u = torch.tensor(X1, dtype=torch.float32)
t_c = torch.tensor(Y1, dtype=torch.float32)

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

scaler = StandardScaler()
t_un_train = scaler.fit_transform(t_u_train)
t_un_val = scaler.transform(t_u_val)

t_un_train = torch.tensor(t_un_train, dtype=torch.float32)
t_un_val = torch.tensor(t_un_val, dtype=torch.float32)

t_cn_train = scaler.fit_transform(t_c_train.reshape(-1, 1))
t_cn_val = scaler.transform(t_c_val.reshape(-1, 1))

t_cn_train = torch.tensor(t_cn_train, dtype=torch.float32)
t_cn_val = torch.tensor(t_cn_val, dtype=torch.float32)

t_cn_train = t_cn_train.view(-1, 1)
t_cn_val = t_cn_val.view(-1, 1)

print(t_un_train.shape)
print(t_cn_train.shape)
print(t_un_val.shape)
print(t_cn_val.shape)

torch.Size([436, 11])
torch.Size([436, 1])
torch.Size([109, 11])
torch.Size([109, 1])


In [439]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                   t_c_train, t_c_val):
  for epoch in range(1, n_epochs + 1):
    t_p_train = model(t_u_train)
    loss_train = loss_fn(t_p_train, t_c_train)

    t_p_val = model(t_u_val)
    loss_val = loss_fn(t_p_val, t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0:
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
            f" Validation loss {loss_val.item():.4f}")

In [440]:
linear_model = nn.Linear(t_u.shape[1], 1)
optimizer = torch.optim.SGD(linear_model.parameters(), lr=1e-3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_cn_train,
    t_c_val = t_cn_val)

Epoch 1, Training loss 0.9152, Validation loss 1.2374
Epoch 1000, Training loss 0.3379, Validation loss 0.5607
Epoch 2000, Training loss 0.3301, Validation loss 0.5669
Epoch 3000, Training loss 0.3295, Validation loss 0.5684
Epoch 4000, Training loss 0.3294, Validation loss 0.5685
Epoch 5000, Training loss 0.3294, Validation loss 0.5685


In [441]:
from collections import OrderedDict

# seq_model = nn.Sequential(
#             nn.Linear(t_u.shape[1], 32),
#             nn.Tanh(),
#             nn.Linear(32, 1))

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(t_u.shape[1], 32)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(32, 1))
]))

In [442]:
optimizer = torch.optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_cn_train,
    t_c_val = t_cn_val)

Epoch 1, Training loss 0.9061, Validation loss 1.6057
Epoch 1000, Training loss 0.3764, Validation loss 0.7174
Epoch 2000, Training loss 0.3535, Validation loss 0.6807
Epoch 3000, Training loss 0.3434, Validation loss 0.6619
Epoch 4000, Training loss 0.3376, Validation loss 0.6488
Epoch 5000, Training loss 0.3336, Validation loss 0.6387


Learning Rate of Linear Model From Problem 5:
- Learning Rate:  0.1
- Epoch 1000, Loss 3.785644
- Epoch 2000, Loss 3.406405
- Epoch 3000, Loss 3.344335
- Epoch 4000, Loss 3.249141
- Epoch 5000, Loss 3.106353

Even when using a smaller learning rate, the neural network model is much faster at deriving a small loss than the linear model is at the same number of iterations. This is a result of the neural network's more complicated model that uses comparisons to the previous output to form a conclusion on the accuracy of its current theta values. The more complicated model also increases the number of training parameters, with 32 times more complexity than the model uses in Assignment 5.

1b. We will increase the network complexity by adding two additional hidden layers, the hidden layers overall. My suggestions for the size of layers are: 32, 64, and 16 respectively. Please redesign the network and compare your training loss value and validation results against the linear regression you implemented in Assignment 5 and Problem 1.a. Can you compare your model complexity? Note: Use the same 20%, and 80% split for training and validation.

In [443]:
size_hl = [32,64,16]

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear_1', nn.Linear(t_u.shape[1], size_hl[0])),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear_2', nn.Linear(size_hl[0], size_hl[1])),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear_3', nn.Linear(size_hl[1], size_hl[2])),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(size_hl[2], 1))
]))

In [444]:
optimizer = torch.optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_cn_train,
    t_c_val = t_cn_val)

Epoch 1, Training loss 1.0390, Validation loss 1.7314
Epoch 1000, Training loss 0.3579, Validation loss 0.6772
Epoch 2000, Training loss 0.3375, Validation loss 0.6272
Epoch 3000, Training loss 0.3327, Validation loss 0.6133
Epoch 4000, Training loss 0.3301, Validation loss 0.6067
Epoch 5000, Training loss 0.3281, Validation loss 0.6025


2a. Create a fully connected Neural Network for all 10 classes in CIFAR-10 with only one hidden layer with a size of 512. Report your training time, training loss, and evaluation accuracy. Analyze your results in your report. (25pt)

In [445]:
from torchvision import datasets, transforms
datapath = './data'
train_data = datasets.CIFAR10(datapath, train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.CIFAR10(datapath, train=False, download=True, transform=transforms.ToTensor())
print(train_data)
print(test_data)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [446]:
import datetime as dt

def Image_Training_Loop(n_epochs, optimizer, model, loss_fn, train_loader):
  #loss_history = []
  loss = 0
  start_time = dt.datetime.now()
  print(f"Training started at {start_time}")
  for epoch in range(1, n_epochs + 1):
    for images, labels in train_loader:
      batch_size = images.shape[0]
      outputs = model(images.view(batch_size, -1))
      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      #print(f"Epoch {epoch}, Loss {loss.item():.4f}")
    #loss_history.append(loss.item())

  print(f"Final Loss: {loss.item():.4f}")
  end_time = dt.datetime.now()
  print(f"Training finished at {end_time}")
  print(f"Training time: {end_time - start_time}")

In [447]:
class_names = train_data.classes
print(class_names)

seq_model_2 = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, 10),
    nn.LogSoftmax(dim=1))

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

optimizer = torch.optim.SGD(seq_model_2.parameters(), lr=1e-3)

Image_Training_Loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = seq_model_2,
    loss_fn = nn.NLLLoss(),
    train_loader = train_loader)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Training started at 2024-08-03 00:00:04.239613
Final Loss: 1.5907
Training finished at 2024-08-03 00:32:30.127977
Training time: 0:32:25.888364


In [448]:
val_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for images, labels in val_loader:
    batch_size = images.shape[0]
    outputs = seq_model_2(images.view(batch_size, -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 44.03%



  2b. Extend your network with two more additional hidden layers, like the example we did in the lecture (pick the sizes of hidden layers properly). Train your network. Report your training time, loss, and evaluation accuracy after 300 epochs. Analyze your results in your report and compare your model size and accuracy over the baseline implementation in Problem 2.a. Do you see any over-fitting? Can you compare your model complexity against problem 2.a? (35pt)


In [449]:
seq_model_3 = nn.Sequential(OrderedDict([
    ('hidden_linear_1', nn.Linear(3072, 512)),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear_2', nn.Linear(512, 256)),
    ('hidden_activation', nn.Tanh()),
    ('hidden_linear_3', nn.Linear(256, 128)),
    ('output_linear', nn.Linear(128, 10)),
    ('hidden_activation', nn.LogSoftmax(dim=1))
]))

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

optimizer = torch.optim.SGD(seq_model_3.parameters(), lr=1e-3)

Image_Training_Loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = seq_model_3,
    loss_fn = nn.NLLLoss(),
    train_loader = train_loader)

Training started at 2024-08-03 01:08:09.890572
Final Loss: nan
Training finished at 2024-08-03 02:45:01.623011
Training time: 1:36:51.732439


In [450]:
val_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
  for images, labels in val_loader:
    batch_size = images.shape[0]
    outputs = seq_model_3(images.view(batch_size, -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print(f"Accuracy: {100 * correct / total:.2f}%")

Accuracy: 10.00%
